#  ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

In [2]:
# Load the data:
data_pok = pd.read_csv('pokemon.txt')
data_pok.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
data_pok = data_pok.drop('#', axis=1)
data_pok.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [6]:
Type_1 =list(data_pok['Type 1'].unique())
Type_1

['Grass',
 'Fire',
 'Water',
 'Bug',
 'Normal',
 'Poison',
 'Electric',
 'Ground',
 'Fairy',
 'Fighting',
 'Psychic',
 'Rock',
 'Ghost',
 'Ice',
 'Dragon',
 'Dark',
 'Steel',
 'Flying']

In [7]:
Type_2 =list(data_pok['Type 2'].unique())
Type_2

['Poison',
 nan,
 'Flying',
 'Dragon',
 'Ground',
 'Fairy',
 'Grass',
 'Fighting',
 'Psychic',
 'Steel',
 'Ice',
 'Rock',
 'Dark',
 'Water',
 'Electric',
 'Fire',
 'Ghost',
 'Bug',
 'Normal']

In [8]:
unique_types = pd.concat([pd.Series(Type_1), pd.Series(Type_2)]).drop_duplicates().reset_index(drop=True)

In [9]:
unique_types = unique_types.dropna()
unique_types

0        Grass
1         Fire
2        Water
3          Bug
4       Normal
5       Poison
6     Electric
7       Ground
8        Fairy
9     Fighting
10     Psychic
11        Rock
12       Ghost
13         Ice
14      Dragon
15        Dark
16       Steel
17      Flying
dtype: object

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`. Be sure to loop through BOTH `Type 1` and `Type 2` to cover all occurrances of each unique type.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [10]:
type_1_sum = data_pok.groupby('Type 1').size().reset_index(name='typ1_total')
type_1_sum

,Type 1,typ1_total
0,Bug,69
1,Dark,31
2,Dragon,32
3,Electric,44
4,Fairy,17
5,Fighting,27
6,Fire,52
7,Flying,4
8,Ghost,32
9,Grass,70


In [11]:
type_2_sum = data_pok.groupby('Type 2').size().reset_index(name='typ2_total')
type_2_sum

,Type 2,typ2_total
0,Bug,3
1,Dark,20
2,Dragon,18
3,Electric,6
4,Fairy,23
5,Fighting,26
6,Fire,12
7,Flying,97
8,Ghost,14
9,Grass,25


In [15]:
pokemon_totals = []

for type_val in unique_types:
    if type(type_val) == str:  # for NaN values
        total = data_pok[data_pok['Type 1'] == type_val]['Total'].append(data_pok[data_pok['Type 2'] == type_val]['Total'])
#         total += pokemon[pokemon['Type 2'] == type_val]['Total'].sum()
        pokemon_totals.append(total)

#print(pokemon_totals)

/var/folders/75/lxv2pt816y3g7mh_b0zy07mw0000gn/T/ipykernel_41157/261654797.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total = data_pok[data_pok['Type 1'] == type_val]['Total'].append(data_pok[data_pok['Type 2'] == type_val]['Total'])
/var/folders/75/lxv2pt816y3g7mh_b0zy07mw0000gn/T/ipykernel_41157/261654797.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total = data_pok[data_pok['Type 1'] == type_val]['Total'].append(data_pok[data_pok['Type 2'] == type_val]['Total'])
/var/folders/75/lxv2pt816y3g7mh_b0zy07mw0000gn/T/ipykernel_41157/261654797.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total = data_pok[data_pok['Type 1'] == type_val]['Total'].append(data_pok[data_pok['Type 2'] == type_val]['Total'])
/

In [14]:
len(pokemon_totals)

18

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [16]:
st.f_oneway(*pokemon_totals)

F_onewayResult(statistic=6.617538296005535, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?

We can see that the p-value is very small, (less than the commonly used significance level of 0.05). This indicates strong evidence against the null hypothesis, suggesting that there is a significant difference between the means of the groups being compared.
